In [2]:
#2 possible ways to get text below in the 2 methods
#Other possibly useful links:
    #https://blog.apilayer.com/build-your-own-resume-parser-using-python-and-nlp/
    #https://www.geeksforgeeks.org/pdf-redaction-using-python/



import PyPDF2
import spacy
from spacy.matcher import Matcher
from spacy.lang.en.examples import sentences

# Code from https://www.geeksforgeeks.org/working-with-pdf-files-in-python/
def read_pdf(filename):
    # creating a pdf file object
    pdfFileObj = open(filename, 'rb')

    # creating a pdf reader object
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

    # printing number of pages in pdf file
    print(pdfReader.numPages)

    # creating a page object
    pageObj = pdfReader.getPage(0)

    # extracting text from page
    print(pageObj.extractText())

    #to save the text and run NLP instead of print
    resume_text = pageObj.extractText()

    # closing the pdf file object
    pdfFileObj.close()

#code from - https://stackoverflow.com/questions/55220455/convert-from-pdf-to-text-lines-and-words-are-broken
def extract_with_pdf_miner():
    import io
    from io import StringIO
    from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
    from pdfminer.converter import TextConverter
    from pdfminer.layout import LAParams
    from pdfminer.pdfpage import PDFPage
    import os
    import sys, getopt

    # converts pdf, returns its text content as a string
    def convert(fname, pages=None):
        if not pages:
            pagenums = set()
        else:
            pagenums = set(pages)

        output = io.StringIO()
        manager = PDFResourceManager()
        converter = TextConverter(manager, output, laparams=LAParams())
        interpreter = PDFPageInterpreter(manager, converter)

        infile = open(fname, 'rb')
        for page in PDFPage.get_pages(infile, pagenums):
            interpreter.process_page(page)
        infile.close()
        converter.close()
        text = output.getvalue()
        output.close
        return text

        # converts all pdfs in directory pdfDir, saves all resulting txt files to txtdir

    def convertMultiple(pdfDir, txtDir):
        if pdfDir == "": pdfDir = os.getcwd() + "\\"  # if no pdfDir passed in
        for pdf in os.listdir(pdfDir):  # iterate through pdfs in pdf directory
            fileExtension = pdf.split(".")[-1]
            if fileExtension == "pdf":
                pdfFilename = pdfDir + pdf
                text = convert(pdfFilename)  # get string of text content of pdf
                textFilename = txtDir + pdf + ".txt"
                textFile = open(textFilename, "w")  # make text file
                textFile.write(text)  # write text to text file

    # set paths accordingly:
    pdfDir = "./"
    txtDir = "./"
    convertMultiple(pdfDir, txtDir)

# Press the green button in the gutter to run the script.
if __name__ == '__main__':
    pass

In [65]:
import spacy
from spacy.matcher import Matcher
import io
from io import StringIO
import os
import sys, getopt
import re

extract_with_pdf_miner()
f = open("./sample_cover_letter.pdf.txt")
text = f.read()

#f = open("./names.txt") # Breaks here, Jupyter crashes
#text = f.read()

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

pattern = [{"LIKE_EMAIL": True}] # Email Address Pattern
matcher.add("email_address", [pattern])

# Process whole documents
text = ("My email is ma@gmail.com. My phone number is 303-303-3033. I live at 123 Main Street. It was another early sunset on a rainy day in Seattle. Andrew was walking with a paper bag of groceries back to his downtown studio apartment at 400 Main Street. He had gotten government permission to stay in this apartment as part of the Upbeat program, focusing on bettering drug-affected neighborhoods in King County. It was an artist supplement program, and Andrew was a published haiku poet and teacher of haiku in public education and sometimes colleges. He had to attend an audition for artistic achievements to be able to get the subsidy. Though it was a miracle to live in downtown Seattle for $800 a month, it did have its downsides, with junkies visiting the weekly exhibitions of his haiku and accompanying minimalist art made by his neighbor, Patrick.")

doc = nlp(text)
matches = matcher(doc)

print("Matches:", [doc[start:end].text for match_id, start, end in matches])

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
count = 0
for entity in doc.ents:
    print(entity.text, entity.label_)
    if(entity.label_ == "PERSON"):
        count+=1
print(count)    

Matches: ['ma@gmail.com']
Noun phrases: ['My email', 'ma@gmail.com', 'My phone number', 'I', '123 Main Street', 'It', 'another early sunset', 'a rainy day', 'Seattle', 'Andrew', 'a paper bag', 'groceries', 'his downtown studio apartment', '400 Main Street', 'He', 'government permission', 'this apartment', 'part', 'the Upbeat program', 'drug-affected neighborhoods', 'King County', 'It', 'an artist supplement program', 'Andrew', 'a published haiku poet', 'teacher', 'haiku', 'public education', 'sometimes colleges', 'He', 'an audition', 'artistic achievements', 'the subsidy', 'it', 'a miracle', 'downtown Seattle', 'it', 'its downsides', 'junkies', 'the weekly exhibitions', 'his haiku', 'accompanying minimalist art', 'his neighbor', 'Patrick']
Verbs: ['live', 'walk', 'get', 'stay', 'focus', 'better', 'affect', 'publish', 'have', 'attend', 'get', 'live', 'have', 'visit', 'accompany', 'make']
303 CARDINAL
Seattle GPE
400 CARDINAL
Main Street FAC
Upbeat ORG
King County GPE
Andrew PERSON
Seatt

In [70]:
lst_entities = {
    'FAC': 'ADDRESS',
    'GPE': 'LOCATION',
    'PERSON': 'PERSON',
    'CARDINAL': 'LOCATION',
    'DATE': 'DATE'
}
entities = {}
for entity in doc.ents:
    entities[entity.text] = entity.label_
new_text = text
pronouns = {"he": "THEY",
                      "him": "THEM",
                      "his": "THEIR",
                      "she": "THEY",
                      "her": "THEM",
                      "hers": "THEIR",
                      "He": "THEY",
                      "His": "THEIR",
                      "She": "THEY",
                      "Her": "THEM",
                     }
persons = ["man","Man","woman","Woman","non-binary","Non-binary"]
def replace(match):
    return pronouns[match.group(0)]
for i in range(0,len(doc)):
    for j in range(i+1,len(doc)):
        token = doc[i:j]
        if(token.text in entities):
            if(entities[token.text] in lst_entities):
                new_text = new_text.replace(token.text,"[" + lst_entities[entities[token.text]] + "]")
            else:
                pass
        elif(token.text in persons):
            new_text = new_text.replace(token.text,"[PERSON]")
        elif(token.text in pronouns):
            new_text = new_text.replace(token.text,pronouns[token.text])
        else:
            for match_id, start, end in matches: # Just emails here for now
                if(doc[start:end].text==token.text):
                    new_text = new_text.replace(token.text,"[REDACTED]")
                    
            PHONE_NUMBER = "\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}"
            DATE = "([\d]{1,2}[/-][\d]{1,2}[/-][\d]{4})|((Jan(uary)?|Feb(ruary)?|Mar(ch)?|Apr(il)?|May|Jun(e)?|Jul(y)?|Aug(ust)?|Sep(tember)?|Oct(ober)?|Nov(ember)?|Dec(ember)?)\s?[\d{1,2}]?[,]?\s+\d{4})"

            # From https://spacy.io/usage/rule-based-matching#regex-text
            lst_regex = {"PHONE_NUMBER":PHONE_NUMBER,"DATE":DATE}
            for regex in lst_regex:
                for match in re.finditer(lst_regex[regex], doc.text):
                    start, end = match.span()
                    span = doc.char_span(start, end)
                    if span is not None:
                        new_text = new_text.replace(span.text,"[" + str(regex) + "]")
            regex = '|'.join(r'\b%s\b' % re.escape(s) for s in pronouns)
            new_txt = re.sub(regex, replace, new_text)
print(new_text)

My email is [REDACTED]. My phone number is [PHONE_NUMBER]. I live at 123 [ADDRESS]. It was another early sunset on a rainy day in [LOCATION]. [PERSON] was walking with a paper bag of groceries back to THEIR downtown studio apartment at [LOCATION] [ADDRESS]. THEY had gotten government permission to stay in tTHEIR apartment as part of the Upbeat program, focusing on bettering drug-affected neighborhoods in [LOCATION]. It was an artist supplement program, and [PERSON] was a published haiku poet and teacher of haiku in public education and sometimes colleges. THEY had to attend an audition for artistic achievements to be able to get the subsidy. Though it was a miracle to live in downtown [LOCATION] for $800 a month, it did have its downsides, with junkies visiting the [DATE] exhibitions of THEIR haiku and accompanying minimalist art made by THEIR neighbor, Patrick.
